In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F
import re
import time

# check what tables are available on ddp

In [2]:
ddp_steer = sqlContext.sql("show tables in ddp_steer")
tables_list = ddp_steer.select('tableName').toPandas()['tableName']
tables = [t for t in tables_list if 'cos_ucl_' in t]
print(tables)

[u'cos_ucl_a_brexecutionresult', u'cos_ucl_a_credit', u'cos_ucl_advice', u'cos_ucl_a_existingcredit', u'cos_ucl_agreement', u'cos_ucl_amortizationsimulation', u'cos_ucl_a_pledgerow', u'cos_ucl_applicant', u'cos_ucl_applicantstatus', u'cos_ucl_applicationhousehold', u'cos_ucl_b_brexecutionresult', u'cos_ucl_b_credit', u'cos_ucl_b_existingcredit', u'cos_ucl_b_pledgerow', u'cos_ucl_calculatedprice', u'cos_ucl_case', u'cos_ucl_casestatus', u'cos_ucl_c_brexecutionresult', u'cos_ucl_c_credit', u'cos_ucl_c_existingcredit', u'cos_ucl_checkatdisbursement', u'cos_ucl_checklist', u'cos_ucl_collateral', u'cos_ucl_connectedcollateralid', u'cos_ucl_controlofcollateral', u'cos_ucl_c_pledgerow', u'cos_ucl_creditcompilation', u'cos_ucl_creditdecision', u'cos_ucl_customer', u'cos_ucl_customerproduct', u'cos_ucl_d_brexecutionresult', u'cos_ucl_d_credit', u'cos_ucl_decisionmaker', u'cos_ucl_deedrow', u'cos_ucl_d_existingcredit', u'cos_ucl_d_pledgerow', u'cos_ucl_e_existingcredit', u'cos_ucl_e_pledgerow', 

# check empty tables

In [3]:
nonempty=[]
empty=[]

for table in tables:
    #print(table)
     # read the table from ddp_steer
    sql_str = "SELECT * FROM ddp_steer."+str(table)
    df=sqlContext.sql(sql_str)
     # check if the table is empty
    if df.count()>0:
        #df.printSchema()
        #df.show(5)
        nonempty.append(table)
    else:
        #print(table+' has no row')
        empty.append(table)

In [4]:
print(len(nonempty),nonempty)

(24, [u'cos_ucl_applicant', u'cos_ucl_applicantstatus', u'cos_ucl_b_brexecutionresult', u'cos_ucl_b_existingcredit', u'cos_ucl_casestatus', u'cos_ucl_c_brexecutionresult', u'cos_ucl_creditdecision', u'cos_ucl_customer', u'cos_ucl_d_credit', u'cos_ucl_decisionmaker', u'cos_ucl_e_existingcredit', u'cos_ucl_household', u'cos_ucl_i_pledgerow', u'cos_ucl_offeraction', u'cos_ucl_owner', u'cos_ucl_pledger', u'cos_ucl_racinputapplicant', u'cos_ucl_repaymentabilitycustomerdata', u'cos_ucl_systemrecommendation', u'cos_ucl_ucindividualgroup', u'cos_ucl_ucindividualreport', u'cos_ucl_ucindividualterm', u'cos_ucl_uctemplateinputparameter', u'cos_ucl_valueestimationoftgcl'])


In [5]:
print(len(empty),empty)

(44, [u'cos_ucl_a_brexecutionresult', u'cos_ucl_a_credit', u'cos_ucl_advice', u'cos_ucl_a_existingcredit', u'cos_ucl_agreement', u'cos_ucl_amortizationsimulation', u'cos_ucl_a_pledgerow', u'cos_ucl_applicationhousehold', u'cos_ucl_b_credit', u'cos_ucl_b_pledgerow', u'cos_ucl_calculatedprice', u'cos_ucl_case', u'cos_ucl_c_credit', u'cos_ucl_c_existingcredit', u'cos_ucl_checkatdisbursement', u'cos_ucl_checklist', u'cos_ucl_collateral', u'cos_ucl_connectedcollateralid', u'cos_ucl_controlofcollateral', u'cos_ucl_c_pledgerow', u'cos_ucl_creditcompilation', u'cos_ucl_customerproduct', u'cos_ucl_d_brexecutionresult', u'cos_ucl_deedrow', u'cos_ucl_d_existingcredit', u'cos_ucl_d_pledgerow', u'cos_ucl_e_pledgerow', u'cos_ucl_f_existingcredit', u'cos_ucl_f_pledgerow', u'cos_ucl_g_pledgerow', u'cos_ucl_h_pledgerow', u'cos_ucl_idcheck', u'cos_ucl_listprice', u'cos_ucl_manualinterestrateucl', u'cos_ucl_newdeed', u'cos_ucl_offer', u'cos_ucl_ongoingcredit', u'cos_ucl_otheraccommodation', u'cos_ucl_oth

# check empty columns and the values in the columns for the nonempty tables

In [53]:
i=0
index=[]
table_name=[]
col_name=[]
col_status=[]
for table in nonempty:
    print('test table: %s' %table)
    #read the table from ddp_steer
    sql_str="select * from ddp_steer."+str(table)
    df=sqlContext.sql(sql_str)
    print('number of rows: %d' %df.count())
    print('number of columns: %d' %len(df.columns))
    for col in df.columns:
        i=i+1
        index.append(i)
        table_name.append(table)
        col_name.append(col)
        print('test column: %s' %col)
        if df.filter(F.col(col).isNull()).count()==0:
            col_status.append('fully populated')
            df.select(col).show(5,False)
        elif (df.filter(F.col(col).isNull()).count()>0) and (df.filter(F.col(col).isNull()).count()<df.count()):
            col_status.append('partlypopulated')
            df.select(col).filter(F.col(col).isNotNull()).show(5,False)
        elif df.filter(F.col(col).isNull()).count()==df.count():
            col_status.append('empty')

test table: cos_ucl_applicant
number of rows: 33
number of columns: 22
test column: COS_UCL_APPLICANT_PERIOD_SRC_STREAM_ID
+-----------------------------------------------------------------------------------------------+
|COS_UCL_APPLICANT_PERIOD_SRC_STREAM_ID                                                         |
+-----------------------------------------------------------------------------------------------+
|COSUCL001                                                                                      |
|I am aware that the bank may request additional information in order to process my application.|
|I agree that the bank performs a credit check.                                                 |
|COSUCL001                                                                                      |
|I am aware that the bank may request additional information in order to process my application.|
+--------------------------------------------------------------------------------------------

In [54]:
pd.DataFrame(list(zip(index, table_name, col_name, col_status)),columns=['ID','table_name','col_name','col_status']).head()

,ID,table_name,col_name,col_status
0,1,cos_ucl_applicant,COS_UCL_APPLICANT_PERIOD_SRC_STREAM_ID,fully populated
1,2,cos_ucl_applicant,COS_UCL_APPLICANT_PERIOD_BATCH_ID,partlypopulated
2,3,cos_ucl_applicant,COS_UCL_APPLICANT_PERIOD_CREATE_DATE,partlypopulated
3,4,cos_ucl_applicant,COS_UCL_APPLICANT_PERIOD_TIME_ZONE,partlypopulated
4,5,cos_ucl_applicant,COS_UCL_APPLICANT_PERIOD_SEQ_NUM,partlypopulated


In [55]:
import csv
w = csv.writer(open("check_cos_ucl_output.csv", "w"))
for i in range(0,len(index)):
    w.writerow([index[i], table_name[i], col_name[i], col_status[i]])

# test table relationships

## rename the column names

In [78]:
def rename(table):    
#    for table in tables:
    # read the table from ddp_steer
    print(table)
    sql_str = "SELECT * FROM ddp_steer."+str(table)
    df=sqlContext.sql(sql_str)

    for col in df.columns:
        df=df.withColumnRenamed(col, col.replace(table.upper()+str('_PERIOD_'),""))
    return df

In [87]:
for i in range(0,len(tables)):
    if i == 0:
        df = rename(tables[i])
        dff = df
    else:
        df = rename(tables[i])
        dff=dff.join(df,['SRC_STREAM_ID','BATCH_ID','CREATE_DATE','TIME_ZONE','SEQ_NUM','CASE_SEQ_NUM'],'left')

cos_ucl_a_brexecutionresult
cos_ucl_a_credit
cos_ucl_advice
cos_ucl_a_existingcredit
cos_ucl_agreement
cos_ucl_amortizationsimulation
cos_ucl_a_pledgerow
cos_ucl_applicant
cos_ucl_applicantstatus
cos_ucl_applicationhousehold
cos_ucl_b_brexecutionresult
cos_ucl_b_credit
cos_ucl_b_existingcredit
cos_ucl_b_pledgerow
cos_ucl_calculatedprice
cos_ucl_case
cos_ucl_casestatus
cos_ucl_c_brexecutionresult
cos_ucl_c_credit
cos_ucl_c_existingcredit
cos_ucl_checkatdisbursement
cos_ucl_checklist
cos_ucl_collateral
cos_ucl_connectedcollateralid
cos_ucl_controlofcollateral
cos_ucl_c_pledgerow
cos_ucl_creditcompilation
cos_ucl_creditdecision
cos_ucl_customer
cos_ucl_customerproduct
cos_ucl_d_brexecutionresult
cos_ucl_d_credit
cos_ucl_decisionmaker
cos_ucl_deedrow
cos_ucl_d_existingcredit
cos_ucl_d_pledgerow
cos_ucl_e_existingcredit
cos_ucl_e_pledgerow
cos_ucl_f_existingcredit
cos_ucl_f_pledgerow
cos_ucl_g_pledgerow
cos_ucl_household
cos_ucl_h_pledgerow
cos_ucl_idcheck
cos_ucl_i_pledgerow
cos_ucl_listpr

In [88]:
#check duplicate var names
print('Total number of features: ', np.size(dff.columns))
print('Duplicate features:')
unique, index = np.unique(dff.columns, axis=0, return_counts=True)
dup = unique[index > 1]
print(dup)

('Total number of features: ', 1257)
Duplicate features:
['APPLICANT_SEQ_NUM' 'APPLICATIONHOUSEHOLD_SEQ_NUM'
 'BREXECUTIONRESULT_ADJUSTEDBYPID' 'BREXECUTIONRESULT_ADJUSTEDDATETIME'
 'BREXECUTIONRESULT_ADJUSTMENTCOMMENT'
 'BREXECUTIONRESULT_ADJUSTMENTMEASURES'
 'BREXECUTIONRESULT_ADJUSTMENTMEASURESCODE'
 'BREXECUTIONRESULT_ADJUSTMENTRESULT'
 'BREXECUTIONRESULT_BANKRULEIDENTIFIER' 'BREXECUTIONRESULT_EXECUTIONRESULT'
 'BREXECUTIONRESULT_ISACTIVE' 'BREXECUTIONRESULT_RULECATEGORY'
 'BREXECUTIONRESULT_RULEDESCRIPTION' 'BREXECUTIONRESULT_RULEID'
 'BREXECUTIONRESULT_SEQ_NUM' 'BREXECUTIONRESULT_VALIDATIONDATETIME'
 'COLLATERAL_SEQ_NUM' 'CREDITDECISION_SEQ_NUM'
 'CREDIT_CREDITAMOUNT_CURRENCYCODE' 'CREDIT_CREDITAMOUNT_VALUE'
 'CREDIT_CREDITOR' 'CREDIT_INTERESTRATE'
 'CREDIT_RACINPUTCREDITTYPEINFORMATION_RACINPUTSTRAIGHTCREDIT_AMORTISATIONAMOUNTPERYEAR_CURRENCYCODE'
 'CREDIT_RACINPUTCREDITTYPEINFORMATION_RACINPUTSTRAIGHTCREDIT_AMORTISATIONAMOUNTPERYEAR_VALUE'
 'CREDIT_RACINPUTCREDITTYPEINFORMATION

In [117]:
nonempty

[u'cos_ucl_applicant',
 u'cos_ucl_applicantstatus',
 u'cos_ucl_b_brexecutionresult',
 u'cos_ucl_b_existingcredit',
 u'cos_ucl_casestatus',
 u'cos_ucl_c_brexecutionresult',
 u'cos_ucl_creditdecision',
 u'cos_ucl_customer',
 u'cos_ucl_d_credit',
 u'cos_ucl_decisionmaker',
 u'cos_ucl_e_existingcredit',
 u'cos_ucl_household',
 u'cos_ucl_i_pledgerow',
 u'cos_ucl_offeraction',
 u'cos_ucl_owner',
 u'cos_ucl_pledger',
 u'cos_ucl_racinputapplicant',
 u'cos_ucl_repaymentabilitycustomerdata',
 u'cos_ucl_systemrecommendation',
 u'cos_ucl_ucindividualgroup',
 u'cos_ucl_ucindividualreport',
 u'cos_ucl_ucindividualterm',
 u'cos_ucl_uctemplateinputparameter',
 u'cos_ucl_valueestimationoftgcl']

In [111]:
#identify common merge keys for all the tables
mergekey = ['SRC_STREAM_ID','BATCH_ID','CREATE_DATE','TIME_ZONE','SEQ_NUM','CASE_SEQ_NUM','ING_DAY','ING_MONTH','ING_YEAR','NEWLINE']
for i in range(0,len(nonempty)):
    if i == 0:
        df = rename(nonempty[i])
        dff = df
    else:
        df = rename(nonempty[i])  
        dff=dff.join(df,mergekey,'left')

cos_ucl_applicant
cos_ucl_applicantstatus
cos_ucl_b_brexecutionresult
cos_ucl_b_existingcredit
cos_ucl_casestatus
cos_ucl_c_brexecutionresult
cos_ucl_creditdecision
cos_ucl_customer
cos_ucl_d_credit
cos_ucl_decisionmaker
cos_ucl_e_existingcredit
cos_ucl_household
cos_ucl_i_pledgerow
cos_ucl_offeraction
cos_ucl_owner
cos_ucl_pledger
cos_ucl_racinputapplicant
cos_ucl_repaymentabilitycustomerdata
cos_ucl_systemrecommendation
cos_ucl_ucindividualgroup
cos_ucl_ucindividualreport
cos_ucl_ucindividualterm
cos_ucl_uctemplateinputparameter
cos_ucl_valueestimationoftgcl


In [113]:
#check duplicate var names
print('Total number of features: ', np.size(dff.columns))
print('Duplicate features:')
unique, index = np.unique(dff.columns, axis=0, return_counts=True)
dup = unique[index > 1]
print(dup)

('Total number of features: ', 236)
Duplicate features:
['APPLICANT_SEQ_NUM' 'APPLICATIONHOUSEHOLD_SEQ_NUM'
 'BREXECUTIONRESULT_ADJUSTEDBYPID' 'BREXECUTIONRESULT_ADJUSTEDDATETIME'
 'BREXECUTIONRESULT_ADJUSTMENTCOMMENT'
 'BREXECUTIONRESULT_ADJUSTMENTMEASURES'
 'BREXECUTIONRESULT_ADJUSTMENTMEASURESCODE'
 'BREXECUTIONRESULT_ADJUSTMENTRESULT'
 'BREXECUTIONRESULT_BANKRULEIDENTIFIER' 'BREXECUTIONRESULT_EXECUTIONRESULT'
 'BREXECUTIONRESULT_ISACTIVE' 'BREXECUTIONRESULT_RULECATEGORY'
 'BREXECUTIONRESULT_RULEDESCRIPTION' 'BREXECUTIONRESULT_RULEID'
 'BREXECUTIONRESULT_SEQ_NUM' 'BREXECUTIONRESULT_VALIDATIONDATETIME'
 'COLLATERAL_SEQ_NUM' 'CREDITDECISION_SEQ_NUM' 'CUSTOMER_SEQ_NUM'
 'EXISTINGCREDIT_AGREEMENTID' 'EXISTINGCREDIT_BANKID'
 'EXISTINGCREDIT_CASEINTERNALID' 'EXISTINGCREDIT_CLEARINGNUMBER'
 'EXISTINGCREDIT_KODSTOMME' 'EXISTINGCREDIT_SEQ_NUM' 'HOUSEHOLD_SEQ_NUM'
 'REPAYMENTABILITYCALCULATION_SEQ_NUM' 'UCINDIVIDUALGROUP_SEQ_NUM'
 'UCINDIVIDUALREPORT_SEQ_NUM']


In [99]:
dff.columns

['SRC_STREAM_ID',
 'BATCH_ID',
 'CREATE_DATE',
 'TIME_ZONE',
 'SEQ_NUM',
 'CASE_SEQ_NUM',
 'APPLICATIONHOUSEHOLD_SEQ_NUM',
 'APPLICANT_SEQ_NUM',
 'APPLICANT_APPLICANTROLE',
 'APPLICANT_ASSERTIONTEXT',
 'APPLICANT_CUSTOMERNUMBER',
 'APPLICANT_FIRSTNAME',
 'APPLICANT_MARITALSTATUS',
 'APPLICANT_MONTHLYINCOME_CURRENCYCODE',
 'APPLICANT_MONTHLYINCOME_VALUE',
 'APPLICANT_OCCUPATION',
 'APPLICANT_PHONENUMBER',
 'APPLICANT_SURNAME',
 'ING_YEAR',
 'ING_MONTH',
 'ING_DAY',
 'NEWLINE',
 'HOUSEHOLD_SEQ_NUM',
 'CUSTOMER_SEQ_NUM',
 'APPLICANTSTATUS_SEQ_NUM',
 'APPLICANTSTATUS_STATUS',
 'APPLICANTSTATUS_STATUSDATETIME',
 'ING_YEAR',
 'ING_MONTH',
 'ING_DAY',
 'NEWLINE',
 'HOUSEHOLD_SEQ_NUM',
 'BREXECUTIONRESULT_SEQ_NUM',
 'BREXECUTIONRESULT_ADJUSTEDBYPID',
 'BREXECUTIONRESULT_ADJUSTEDDATETIME',
 'BREXECUTIONRESULT_ADJUSTMENTCOMMENT',
 'BREXECUTIONRESULT_ADJUSTMENTMEASURES',
 'BREXECUTIONRESULT_ADJUSTMENTMEASURESCODE',
 'BREXECUTIONRESULT_ADJUSTMENTRESULT',
 'BREXECUTIONRESULT_BANKRULEIDENTIFIER',
 '

In [162]:
unique_pd=pd.DataFrame(list(zip(unique, index)),columns=['col_name','times'])
unique_pd[(unique_pd['times']>=2) & (unique_pd['times']<=24)].head(20)

,col_name,times
12,APPLICANT_SEQ_NUM,2
14,APPLICATIONHOUSEHOLD_SEQ_NUM,3
16,BREXECUTIONRESULT_ADJUSTEDBYPID,2
17,BREXECUTIONRESULT_ADJUSTEDDATETIME,2
18,BREXECUTIONRESULT_ADJUSTMENTCOMMENT,2
19,BREXECUTIONRESULT_ADJUSTMENTMEASURES,2
20,BREXECUTIONRESULT_ADJUSTMENTMEASURESCODE,2
21,BREXECUTIONRESULT_ADJUSTMENTRESULT,2
22,BREXECUTIONRESULT_BANKRULEIDENTIFIER,2
23,BREXECUTIONRESULT_EXECUTIONRESULT,2
